In [ ]:
import cv2
import face_recognition
import numpy as np
import os

# Load or capture your face
def capture_or_load_user_face(file_path='/Users/badrinaths/Documents/Anti Shoulder Surfing System/My Face.jpg'):
    if os.path.exists(file_path):
        image = face_recognition.load_image_file(file_path)
        encoding = face_recognition.face_encodings(image)
        if encoding:
            return encoding[0]
        else:
            print("❌ Face encoding failed from image.")
            return None
    else:
        cap = cv2.VideoCapture(0)
        print("Press 's' to save your face")
        while True:
            ret, frame = cap.read()
            cv2.imshow("Capture Face", frame)
            if cv2.waitKey(1) & 0xFF == ord('s'):
                cv2.imwrite(file_path, frame)
                break
        cap.release()
        cv2.destroyAllWindows()
        return capture_or_load_user_face(file_path)

# Start Monitoring
def monitor_intrusion(known_encoding):
    cap = cv2.VideoCapture(0)
    triggered = False

    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Frame capture failed.")
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb)
        encodings = face_recognition.face_encodings(rgb, face_locations)

        face_count = len(encodings)
        intruder = False

        for enc in encodings:
            match = face_recognition.compare_faces([known_encoding], enc)[0]
            if not match:
                intruder = True

        if face_count > 1 or intruder:
            if not triggered:
                print("Intruder detected! Showing warning...")
                triggered = True
        else:
            triggered = False

        if triggered:
            cv2.putText(frame, "⚠️ Privacy Alert: Unknown Face Detected!", (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        cv2.imshow("Anti-Shoulder Surfing Monitor", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run Everything
known_face_encoding = capture_or_load_user_face()
if known_face_encoding is not None:
    monitor_intrusion(known_face_encoding)
else:
    print("❌ Known face encoding missing.")


Intruder detected! Showing warning...
Intruder detected! Showing warning...
Intruder detected! Showing warning...
Intruder detected! Showing warning...
Intruder detected! Showing warning...
